In [ ]:
# submit_FLEXPART.ipynb

# Checks all the required meteo files are available.
# If so, checks AVAILABLE, COMMAND, pathnames and RELEASES are in the FLEXPART directory
# And then launches the FLEXPART executable to submit the batch job
# Otherwise warnings are given.

# C. Mackay September 2022 (Catherine.Mackay@aero.obs-mip.fr)
# https://github.com/ckmackay/SOFT-IO-LI.git

# Suggestions/improvements to be made:

# These functions exist in FLEXPART_auto.ipynb and just need to be uncommented it to launch 
# automatically if meteo files are already stored locally.
""" Remarques
    - faire une boucle pour que ça lance FLEXPART pour tous les vols trouvés, pour l'instant juste 1 par 1
<!> Relancer FLEXPART quand j'aurais fait la modif pour quand il y a juste NO ou juste NO2
"""


In [1]:
import os
import os.path
from os.path import exists
import subprocess

In [21]:
# Check paths
flexpart_path = '/o3p/macc/flexpart10.4/flexpart_v10.4_3d7eebf/src/exercises/soft-io-li/'
flight_dir = 'temp2' # dossier du vol donné ########################################## (<!> PAS loop pour tous les lancer à la suite pour l'instant)
meteo_path='/o3p/wolp/ECMWF/ERA5/050deg_1h_T319_eta1/'

In [72]:
# Change directory to FLEXPART directory for given flight, check files to mlaunch FLEXPART are in place and execute flexpart.sh
# Change back to current working directory

def launch_FLEXPART():

    cwd = os. getcwd()
    os.chdir(flexpart_path+flight_dir)

    if (os.path.exists(flexpart_path+flight_dir+'/options/RELEASES'))==True & (os.path.exists(flexpart_path+flight_dir+'/AVAILABLE'))==True & (os.path.exists(flexpart_path+flight_dir+'/pathnames'))==True & (os.path.exists(flexpart_path+flight_dir+'/options/COMMAND'))==True :  
        print("Files for running FLEXPART exist")

        output = subprocess.Popen(['sbatch -p o3pwork '+flexpart_path+flight_dir+"/flexpart.sh"],shell=True)
        print("FLEXPART process submitted to o3pwork queue on nuwa")
    
    os.chdir(cwd)


In [73]:
with open(flexpart_path+flight_dir+'/AVAILABLE', 'r+') as f:
        list_of_lines = f.readlines()
        
ERA5_file=[]
file_info=[]
count = 0
for i in range(3, len(list_of_lines)-1):
    file_info = list_of_lines[i].split(" ")
    ERA5_file=(str(file_info[7]))
    #print(meteo_path+ERA5_file)
    if (os.path.exists(meteo_path+str(ERA5_file)))==True:
        count = count+1
        #print("ERA5 file present")
    else:
        print("***WARNING*** ERA5_file missing", meteo_path+ERA5_file)
        
if count==240:
    print("All meteo files present")
    print("Launching FLEXPART!!!")
    launch_FLEXPART()
else:
    missing = 240-count
    print("WARNING:", missing, "meteo files missing!")